In [1]:
import numpy as np
import os
import glob
from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import multiprocessing  # <--- Importado para paralelismo

In [2]:
# Define input and output directories
in_directory = "./../hsi_airborne/"
out_directory = "./../hsi_airborne/pca/"
npy_files = glob.glob(f"{in_directory}*.npy")

n_files = len(npy_files)


# create out directory if dont exist
if not os.path.exists(out_directory):
    os.makedirs(out_directory)

In [3]:

# Parâmetros do Group-wise PCA
TOTAL_NC = 32          # Número total de componentes principais desejado no final
NUM_GROUPS = 4           # Em quantos grupos as bandas espectrais serão divididas

# Dimensões esperadas de cada patch (bandas, altura, largura)
DIMS = (430, 20, 20)


# --- Verificações Iniciais ---
if TOTAL_NC % NUM_GROUPS != 0:
    raise ValueError("O número total de componentes (TOTAL_NC) deve ser divisível pelo número de grupos (NUM_GROUPS).")
NC_PER_GROUP = TOTAL_NC // NUM_GROUPS





In [17]:
# Função para processar um único arquivo .npy
def process_single_file(data):
    try:
        X = data
        # replace nan with 0
        X = np.nan_to_num(X, nan=0.0)
        if X.shape == DIMS:
            X = np.moveaxis(X, 0, -1)

            h, w, c = X.shape
            
            X = np.reshape(X, (-1, DIMS[0]))
            return X, h, w, c
    except Exception:
        # Ignora arquivos corrompidos ou com erro de leitura
        return None


def split_data(data_list, group=4):
    output_data = data_list
    step = group // 2
    for i in range(step):
        split_data = []
        for data in output_data:
            n, c = data.shape
            data_s1 = data[:, :c // 2]
            data_s2 = data[:, c // 2:]
            split_data.append(data_s1)
            split_data.append(data_s2)
        output_data = split_data
    return output_data


def applyGWPCA(X, nc=32, group=4, whiten=True):
    h, w, c = X.shape
    X = np.reshape(X, (-1, c))
    X = (X - X.min()) / (np.max(X) - np.min(X))

    X_split = split_data([X], group)
    pca_data_list = []
    for i, x in enumerate(X_split):
        pca = PCA(n_components=nc // group, whiten=whiten, random_state=42)
        pca_data = pca.fit_transform(x)
        pca_data_list.append(pca_data)

    out = np.concatenate(pca_data_list, axis=-1)
    out = np.reshape(out, (h, w, -1))
    return out





In [11]:
data = np.load(npy_files[0])
data.shape


(1876, 430, 20, 20)

In [12]:
# incremental pca

# create the four ipca
standardscaler = [StandardScaler() for _ in range(NUM_GROUPS)]
ipcas = [IncrementalPCA(n_components=NC_PER_GROUP, whiten=True) for _ in range(NUM_GROUPS)]


for i in tqdm(range(data.shape[0]), desc="Fitting Incremental PCA"):
    pixel_matrix = process_single_file(data[i])
    if pixel_matrix is not None:
        # Dividir os dados em grupos
        grouped_data = split_data([pixel_matrix], group=NUM_GROUPS)
        for i, group_data in enumerate(grouped_data):
            # Ajustar o IPCA incrementalmente
            ipcas[i].partial_fit(group_data)


Fitting Incremental PCA: 100%|██████████| 1876/1876 [00:37<00:00, 49.40it/s]


In [19]:
# apply the pca tranform in all data

name_file = 0

for i in tqdm(range(data.shape[0]), desc="Transforming and saving files"):
    name_file += 1
    pixel_matrix, h, w, c = process_single_file(data[i])
    
    if pixel_matrix is not None:
        # Dividir os dados em grupos
        grouped_data = split_data([pixel_matrix], group=NUM_GROUPS)
        pca_data_list = []
        for i, group_data in enumerate(grouped_data):
            # Transformar os dados usando o IPCA ajustado
            pca_data = ipcas[i].transform(group_data)
            pca_data_list.append(pca_data)

        # Concatenar os dados PCA de todos os grupos
        out = np.concatenate(pca_data_list, axis=-1)
        out = np.reshape(out, (h, w, -1))
        # Salvar o resultado em um arquivo .npy
        output_file = os.path.join(out_directory, f"pca_{name_file:05d}_DATA.npy")
        np.save(output_file, out)


Transforming and saving files: 100%|██████████| 1876/1876 [00:01<00:00, 1098.86it/s]
